In [1]:
import geopandas as gp
import pandas as pd
import os
import glob

In [2]:
projcrs = 4326

In [3]:
def convert_geopandas(df):
    '''Convert pandas df to geodataframe, drop orig Lat/Long cols, and set crs to common project crs'''
    
    df = gp.GeoDataFrame(
        df, geometry=gp.points_from_xy(df["Longitude"],
                                       df["Latitude"]), 
        crs = projcrs)

    df.drop(['Latitude', 'Longitude'], axis=1, inplace=True) 
    return(df)

In [4]:
def read_excel(sheet_name):
    
    all_files = glob.glob("data/*.xlsx")
    temp = []

    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=0, sheet_name=sheet_name)
        temp.append(df)

    df = pd.concat(temp, axis=0, ignore_index=True)

    # some data cleanup
    df.rename(columns={'BCM 2019':'BCM_2019', 'Avg. temp':'avg_temp',
                       'BCM 2018':'BCM_2018', 'BCM 2020':'BCM_2020',
                       'BCM 2021':'BCM_2021',
                       'Detection_frequency_2012':'Det_freq12',
                       'Detection_frequency_2013':'Det_freq13',
                       'Detection_frequency_2014':'Det_freq14',
                       'Detection_frequency_2015':'Det_freq15',
                       'Detection_frequency_2016':'Det_freq16',
                       'Detection_frequency_2017':'Det_freq17',
                       'Detection frequency 2018':'Det_freq18',
                       'Detection freq. 2019':'Det_freq19',
                       'Detection frequency 2020':'Det_freq20',
                       'Detection frequency 2021':'Det_freq21',
                       'Clear_obs_2012': 'clr_obs12',
                       'Clear_obs_2013': 'clr_obs13',
                       'Clear_obs_2014': 'clr_obs14',
                       'Clear_obs_2015': 'clr_obs15',
                       'Clear_obs_2016': 'clr_obs16',
                       'Clear_obs_2017': 'clr_obs17',
                       'Clear obs 2018': 'clr_obs18',
                       'Clear obs. 2019': 'clr_obs19',
                       'Clear Obs. 2020': 'clr_obs20',
                       'Clear Obs. 2021': 'clr_obs21'}, inplace=True)
    
    df["flare_category"] = sheet_name  # add new column for overall flare type, to make summary stats easier
    
    return(df)

In [5]:
# define sheet names
all_upstream="flares_upstream"
oil_downstream="flares_oil_downstream"
gas_downstream="flares_gas_downstream"

# create dfs by flare category
flares_upstream = read_excel(all_upstream)
oil_downstream = read_excel(oil_downstream)
gas_downstream = read_excel(gas_downstream)

# combine into single df and convert to geodataframe
all_flares = convert_geopandas(pd.concat([flares_upstream, oil_downstream, gas_downstream]))

/Users/natebender/opt/anaconda3/envs/capstonermi37/lib/python3.7/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [6]:
all_flares.to_file("data/all_flares.shp")

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
